In [ ]:
# !pip install python-dotenv twitchAPI

In [ ]:
# https://pytwitchapi.readthedocs.io/en/v2.5.1/modules/twitchAPI.twitch.html?highlight=game%20id#twitchAPI.twitch.Twitch.get_games #doc api
#https://github.com/Teekeks/pyTwitchAPI/blob/v2.5.1/twitchAPI/twitch.py #doc funcoes
#https://dev.twitch.tv/docs/api/reference#get-clips #doc twitch api

In [1]:
import scrapy
from scrapy import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from logzero import logfile, logger
from random import randint

In [2]:
from dotenv import load_dotenv
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
import twitchAPI.types as types

datetime_inicio_str='2022-01-01 00:00:00'
datetime_fim_str='2022-03-31 23:59:59'
datetime_inicio = datetime.strptime(datetime_inicio_str, '%Y-%m-%d %H:%M:%S')
datetime_fim = datetime.strptime(datetime_fim_str, '%Y-%m-%d %H:%M:%S')

#Carrega variaveis de ambiente para parametrização
load_dotenv('vars.env')

from twitchAPI.twitch import Twitch

#Autenticacao
twitch = Twitch(os.environ.get('cli_id'), os.environ.get('cli_token'))

In [ ]:
from twitchAPI.twitch import Twitch
from twitchAPI.oauth import UserAuthenticator
from twitchAPI.types import AuthScope
twitch = Twitch(os.environ.get('cli_id'), os.environ.get('cli_token'))

#Permissão com autenticação de usuario com o scope USER_READ_BROADCAST 
target_scope = [AuthScope.USER_READ_BROADCAST]
auth = UserAuthenticator(twitch, target_scope, force_verify=False)
# this will open your default browser and prompt you with the twitch verification website
token, refresh_token = auth.authenticate()
# add User authentication
twitch.set_user_authentication(token, target_scope, refresh_token)

#Eu só preciso do Scope caso eu defina que minha aplicação precise ter acesso a obtençao de informações sensivel do usuario

# from twitchAPI.twitch import Twitch
# from twitchAPI.oauth import UserAuthenticator
# from twitchAPI.types import AuthScope

# twitch = Twitch(os.environ.get('cli_id'), os.environ.get('cli_tok²‘¡²¡²¡²‘’en'))

# target_scope = [AuthScope.BITS_REA, AuthScope.CHANNEL_SUBSCRIPTIONS]
# auth = UserAuthenticator(twitch, target_scope, force_verify=False)
# # this will open your default browser and prompt you with the twitch verification website
# token, refresh_token = auth.authenticate()
# # add User authentication
# twitch.set_user_authentication(token, target_scope, refresh_token)

#nota: para acessar informações quantidade de usuarios banidos ou com timeout, deve ter o papel de moderaçao do canal twitch.get_banned_users
#nota: para acessar info de bits leaderboard tbm precisa de autencicao de usuario twitch.get_bits_leaderboard
# twitch.get_broadcaster_subscriptions(broadcaster_id='225588135') requer autenticacao
#get_extension_analytics


In [ ]:
#Carga Lista de canais
df = pd.read_csv('DF_STREAM_PT_NOME_IDS.csv', sep=';')

# Total de follows

In [ ]:
df["total_followers"] = None
for index, row in df.iterrows():
    resultado_consulta = twitch.get_users_follows(to_id=row['broadcaster_id'], first=1)
    # df["total_followers"] = np.where(df["broadcaster_id"] == row['broadcaster_id'], resultado_consulta['total'], None)
    df.loc[df.broadcaster_id == row['broadcaster_id'], 'total_followers'] = resultado_consulta['total']
df.to_csv('DF_STREAM_PT_NOME_IDS_wFollowers.csv', mode='w', sep=";", header=True, index=False)
#Gerado em 01/03/2022 14:18

# Time associado

In [ ]:
df["team_id"] = None
for index, row in df.iterrows():
    try:
        resultado_consulta = twitch.get_channel_teams(broadcaster_id=row['broadcaster_id'])
        df.loc[df.broadcaster_id == row['broadcaster_id'], 'team_id'] = int(resultado_consulta['data'][0]['id'])
    except:
        df.loc[df.broadcaster_id == row['broadcaster_id'], 'team_id'] = 0
df.to_csv('DF_STREAM_PT_NOME_IDS_wteam.csv', mode='w', sep=";", header=True, index=False)
#Gerado em 01/03/2022 21:50

## Regras da quantidade de seguidores nos canais de streamming
- Há 4458 stream com 1 follow 
- há 17.452  streamns com menos de 20 seguidores
- # há 34.055   streamns com menos de 50 seguidores
- há 13.244 streamns com menos menos 70% de 50 seguidores(35) que é o minimo do objetivo para se tornar afiliado da twitch
- há 13.997  streamns com menos menos 60% de 50 seguidores(30) que é o minimo do objetivo para se tornar afiliado da twitch
<!-- df.loc[(df.total_followers >= 30)] -->

# Get subscriptions
<i>Não é informação aberta </i>

In [ ]:
# # Eu só preciso do Scope caso eu defina que a aplicação precise ter acesso
# # a obtençao de informações sensivel do usuario que ta acessando minha aplicaçao

# #As informaçoes de get_broadcaster_subscriptions são obtiveis apenas apenas com autorizaçao dos usuario
# #  Requires User authentication with scope :const:`twitchAPI.types.AuthScope.CHANNEL_READ_SUBSCRIPTIONS
# from twitchAPI.twitch import Twitch
# from twitchAPI.oauth import UserAuthenticator
# from twitchAPI.types import AuthScope

# twitch = Twitch(os.environ.get('cli_id'), os.environ.get('cli_token'))

# target_scope = [AuthScope.BITS_READ, AuthScope.CHANNEL_SUBSCRIPTIONS, AuthScope.CHANNEL_READ_SUBSCRIPTIONS]
# auth = UserAuthenticator(twitch, target_scope, force_verify=False)
# # this will open your default browser and prompt you with the twitch verification website
# token, refresh_token = auth.authenticate()
# # add User authentication
# twitch.set_user_authentication(token, target_scope, refresh_token)

# twitch.get_broadcaster_subscriptions(broadcaster_id='225588135') 

# Get Videos

In [ ]:
# from twitchAPI.types import VideoType 

# df_ultima_steam = None
# for index, row in df.iterrows():
#     try:
#         resultado_dict = twitch.get_videos(user_id=row['broadcaster_id'], first=1, video_type=VideoType.ALL)['data'][0]
#         data_resultado_consulta = pd.DataFrame(resultado_dict)
#         if df_ultima_steam == None:
#             data_resultado_consulta.to_csv('ultimos_videos_registrados20220301.csv', mode='w', header=True, sep=';')
#         else:
#             data_resultado_consulta.to_csv('ultimos_videos_registrados20220301.csv', mode='a', header=False, sep=';')
#     except:
#         print('não encontrado:', row['broadcaster_id'])
# #terminou de executar 02/03/2022

# Get Emote Sets

In [ ]:
df = pd.read_csv('DF_STREAM_PT_NOME_IDS_wteam.csv', sep=';')
df["qtd_emotes"] = None
for index, row in df.iterrows():
    try:
        resultado_consulta = twitch.get_channel_emotes(broadcaster_id=row['broadcaster_id'])['data']
        df.loc[df.broadcaster_id == row['broadcaster_id'], 'qtd_emotes'] = int(len(resultado_consulta))
    except:
        df.loc[df.broadcaster_id == row['broadcaster_id'], 'qtd_emotes'] = 0
df.to_csv('DF_STREAM_PT_NOME_IDS_wqtdemotes.csv', mode='w', sep=";", header=True, index=False)

# Get Stream markers
- tem relaçao com os videos

In [ ]:
df = pd.read_csv('DF_STREAM_PT_NOME_IDS_wteam.csv', sep=';')

df_tags = pd.DataFrame({'tag_id': pd.Series(dtype='str'),
                              'is_auto': pd.Series(dtype='boolean'),
                              'localization_names': pd.Series(dtype='str'),
                              'localization_descriptions': pd.Series(dtype='str'),
                              'broadcaster_id': pd.Series(dtype='str')
                              })
df_tags.to_csv(f'canal_tags.csv', mode='w', sep=";", header=True, index=False)
for index, row in df.iterrows():
    try:
        resultado_consulta = twitch.get_stream_tags(broadcaster_id=row['broadcaster_id'])
        data_resultado_consulta = pd.DataFrame(resultado_consulta['data'])

        #Normalizar coluna localization_names
        data_resultado_consulta = data_resultado_consulta.join(pd.DataFrame(data_resultado_consulta.pop('localization_names').values.tolist()))
        for column in data_resultado_consulta.columns.tolist():
            if column not in ['tag_id', 'is_auto', 'localization_names', 'localization_descriptions', 'pt-br']:
                data_resultado_consulta = data_resultado_consulta.drop([column], axis = 1)
        data_resultado_consulta = data_resultado_consulta.rename(columns={"pt-br": "localization_names"})

        #Normalizar coluna localization_descriptions
        data_resultado_consulta = data_resultado_consulta.join(pd.DataFrame(data_resultado_consulta.pop('localization_descriptions').values.tolist()))
        for column in data_resultado_consulta.columns.tolist():
            if column not in ['tag_id', 'is_auto', 'localization_names', 'localization_descriptions', 'pt-br']:
                data_resultado_consulta = data_resultado_consulta.drop([column], axis = 1)
        data_resultado_consulta = data_resultado_consulta.rename(columns={"pt-br": "localization_descriptions"})

        data_resultado_consulta['broadcaster_id'] = row['broadcaster_id']
        data_resultado_consulta.to_csv('canal_tags.csv', mode='a', sep=";", header=False, index=False)
    except:
        pd.DataFrame([row_id.to_dict()]).to_csv('streammers_sem_tags.csv', mode='a', sep=";", header=False, index=False)

In [ ]:
print('oi')

In [ ]:
# from ast import literal_eval

# data_resultado_consulta.localization_names = data_resultado_consulta.localization_names.fillna({i: {} for i in data_resultado_consulta.index})

In [ ]:
import os 
  
# shutdown = input("Do you wish to shutdown your computer ? (yes / no): ") 
  
# if shutdown == 'no': 
#     exit() 
# else: 

os.system("shutdown /s /t 1") 